## モジュールのimport

In [ ]:
# Module
import sys
sys.path.append("../")
from mod.numpyro_utility import *

# DataFrame, Numerical computation
import polars as pl
pl.Config(fmt_str_lengths = 100, tbl_cols = 100, tbl_rows = 100)
import pandas as pd
import numpy as np
import jax
import jax.numpy as jnp

# ベイズ推定
import numpyro
import numpyro.distributions as dist # 確率分布

# plot
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import arviz as az

# plotの設定
import json
def to_rc_dict(dict):
    """
    jsonファイルのdictを読み込む
    """
    return {f'{k1}.{k2}': v for k1,d in dict.items() for k2,v in d.items()}

file_path = "../mod/rcParams.json"
with open(file_path) as f: 
    plt.rcParams.update(to_rc_dict(json.load(f)))

# 日本語 or 英語の2択
import japanize_matplotlib
#plt.rcParams['font.family'] = "Times New Roman"

## 5.2 線形回帰のベイズ推論

### 5.2.1 問題設定
versicolorのがく片の長さ(sepal_length)と幅(sepal_width)の関係を1次関数近似する。

### 5.2.2 データ準備

In [ ]:
# データセットを読み込む
df = sns.load_dataset("iris")

# setosa を抽出する
df_versicolor = df.query('species == "versicolor"')

# sepal_lengthとsepal_widthの列を抽出
X = jnp.array(df_versicolor['sepal_length'], dtype = float)
Y = jnp.array(df_versicolor['sepal_width'], dtype = float)

In [ ]:
plt.title('２つの項目間の関係')
plt.scatter(X, Y, label='ベイズ推論で利用', c='b', marker='o')
plt.legend()
plt.xlabel('sepal_length')
plt.ylabel('sepal_width')

### 5.2.3 確率モデル定義1
この説は省略する。

### 5.2.4 確率モデル2
数式で整理して、プログラミングで実装する。

1. がく片の長さ sepal_length $x$ とがく片の幅 sepal_width $y$ とを1次関数で近似できる。
    * $y_{i} \approx \omega_{0} + \omega_{1} x_{i}$
1. 1次関数の右辺を $\mu_{i}$ とおく。
    * $\mu_{i} \equiv \omega_{0} + \omega_{1} x_{i}$
1. がく片の幅 sepal_width $y$ が正規分布に従うと仮定する。
    * 形状パラメータの $\mu_{i}$ が説明変数によって変化すると仮定する。
    * $y_{i} \sim N(\mu_{i}, \epsilon^2)$
    * 正規分布の標準偏差 $\epsilon$ の情報はないので広めの値を取る
        * $\epsilon \sim HN(10^2)$
1. 切片や係数の情報は無いので広めの値を取る
        * $\omega_{0} \sim N(0, 10^2)$
        * $\omega_{1} \sim N(0, 10^2)$

前節までより少し複雑な順番で実装する。

In [ ]:
def model_linear_regression(X, Y = None):
    '''
        5.2節のversicolorのがく片長さと幅の確率分布モデル
    '''
    # 4.1 $\omega_{0} \sim N(0, 10^2)$
    ω0 = numpyro.sample("ω0", dist.Normal(loc = 0, scale = 10))
    # 4.2 $\omega_{1} \sim N(0, 10^2)$
    ω1 = numpyro.sample("ω1", dist.Normal(loc = 0, scale = 10))
    # 2. 1次関数の右辺を $\mu_{i}$ とおく。
    μ = numpyro.deterministic("μ", ω0 + ω1 * X)
    # 3. 正規分布の標準偏差は $\epsilon \sim HN(10^2)$ と仮定する。
    ε = numpyro.sample("ε", dist.HalfNormal(scale = 10))
    # ベクトル化
    N = len(X)
    with numpyro.plate("N", N):
        # 1. $y_{i} \sim N(\mu, \sigma^2)$
        numpyro.sample("Y", dist.Normal(loc = μ, scale = ε), obs = Y)

In [ ]:
model_args = {
    "X": X,
    "Y": Y
}
try_render_model(model_linear_regression, render_name = "線形回帰", **model_args)

### 5.2.5 サンプリングと結果分析

In [ ]:
model_args = {
    "X": X,
    "Y": Y
}
idata = run_mcmc(
    model_linear_regression,
    num_chains = 4,
    num_warmup = 1000,
    num_samples = 1000,
    thinning = 1,
    seed = 42,
    target_accept_prob = 0.8,
    log_likelihood = False,
    **model_args
)

In [ ]:
az.plot_trace(idata, compact = False, var_names = ["ω0", "ω1", "ε"])
plt.tight_layout()

In [ ]:
ax = az.plot_posterior(idata, var_names = ["ω0", "ω1", "ε"])
plt.suptitle("1次関数近似の係数の分布")
plt.tight_layout()
plt.show()

### 5.2.6 散布図と回帰直線の重ね描き

In [ ]:
# xの２点をNumPy配列にする
x_values = np.array([X.min()-0.1, X.max()+0.1])
print(x_values, x_values.shape)

# サンプリング結果からalphaとbetaを取り出しshapeを加工する
ω0_posterior = idata['posterior']['ω0'].values.reshape(-1, 1)
ω1_posterior = idata['posterior']['ω1'].values.reshape(-1, 1)

# shapeの確認
print(ω0_posterior.shape, ω1_posterior.shape)

# 2000パターンそれぞれで、２点の１次関数値の計算
y_preds = x_values * ω1_posterior + ω0_posterior
print(y_preds.shape)

In [ ]:
for y_pred in y_preds:
    plt.plot(x_values, y_pred, lw=1, alpha=0.01, c='g')
plt.scatter(X, Y)
plt.title('ベイズ推論による回帰直線')
plt.xlabel('sepal_length')
plt.ylabel('sepal_width');

### 5.2.7 少ない観測値でのベイズ推論
省略。